In [87]:
import pandas as pd
import numpy as np
import seaborn as sns

In [88]:
df = pd.read_csv('/content/sales.csv')
df.head()

,orderid,productid,productname,qty,orderdate,unitprice,custid,shipcountry
0,10248,11,Product QMVUN,12,2006-07-04 00:00:00,14.0,85,France
1,10248,42,Product RJVNM,10,2006-07-04 00:00:00,9.8,85,France
2,10248,72,Product GEEOO,5,2006-07-04 00:00:00,34.8,85,France
3,10249,14,Product PWCJB,9,2006-07-05 00:00:00,18.6,79,Germany
4,10249,51,Product APITJ,40,2006-07-05 00:00:00,42.4,79,Germany


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155 entries, 0 to 2154
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   orderid      2155 non-null   int64  
 1   productid    2155 non-null   int64  
 2   productname  2155 non-null   object 
 3   qty          2155 non-null   int64  
 4   orderdate    2155 non-null   object 
 5   unitprice    2155 non-null   float64
 6   custid       2155 non-null   int64  
 7   shipcountry  2155 non-null   object 
dtypes: float64(1), int64(4), object(3)
memory usage: 134.8+ KB


In [90]:
df.shape

(2155, 8)

In [91]:
df = df.loc[df['qty'] > 0]
df.shape

(2155, 8)

In [92]:
df['custid'].describe()

count    2155.000000
mean       47.265893
std        25.847121
min         1.000000
25%        24.000000
50%        47.000000
75%        69.500000
max        91.000000
Name: custid, dtype: float64

In [93]:
df1 = df.dropna(subset=['custid'])
df1.shape

(2155, 8)

In [110]:
customer_matrix = df1.pivot_table(index='custid',columns='productid',values='qty',aggfunc='sum')
customer_matrix.head()

productid,1,2,3,4,5,6,7,8,9,10,...,68,69,70,71,72,73,74,75,76,77
custid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,6.0,NaN,NaN,16.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,15.0,2.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,5.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN
3,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN
4,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,53.0,12.0,NaN,3.0,NaN,NaN,NaN,NaN
5,35.0,21.0,30.0,12.0,NaN,NaN,NaN,NaN,NaN,22.0,...,NaN,NaN,NaN,32.0,NaN,25.0,NaN,51.0,21.0,NaN


In [95]:
customer_matrix.shape

(89, 77)

In [111]:
customer_matrix = customer_matrix.applymap(lambda x: 1 if x>0 else 0)

In [112]:
from sklearn.metrics.pairwise import cosine_similarity
user_to_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_matrix))

In [113]:
user_to_user_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
0,1.000000,0.000000,0.077850,0.123091,0.346977,0.261116,0.301511,0.000000,0.258544,0.174078,...,0.213201,0.440086,0.064282,0.100504,0.064282,0.227921,0.226134,0.319801,0.219382,0.080582
1,0.000000,1.000000,0.163299,0.193649,0.259938,0.182574,0.189737,0.000000,0.325396,0.304290,...,0.111803,0.000000,0.134840,0.421637,0.269680,0.358569,0.237171,0.167705,0.153393,0.084515
2,0.077850,0.163299,1.000000,0.210819,0.339581,0.149071,0.258199,0.316228,0.265684,0.397523,...,0.273861,0.161515,0.275241,0.172133,0.330289,0.243975,0.193649,0.365148,0.250490,0.138013
3,0.123091,0.193649,0.210819,1.000000,0.436251,0.235702,0.326599,0.166667,0.350070,0.314270,...,0.072169,0.255377,0.435194,0.408248,0.348155,0.385758,0.357217,0.252591,0.099015,0.327327
4,0.346977,0.259938,0.339581,0.436251,1.000000,0.284747,0.493197,0.335578,0.451107,0.442940,...,0.174371,0.479914,0.245350,0.219199,0.385550,0.466027,0.369898,0.464991,0.438599,0.307562


In [114]:
user_to_user_sim_matrix.shape

(89, 89)

In [115]:
user_to_user_sim_matrix.loc[12.0].sort_values(ascending = False)

12    1.000000
21    0.365148
13    0.342997
23    0.246183
17    0.235702
        ...   
34    0.000000
33    0.000000
32    0.000000
31    0.000000
88    0.000000
Name: 12, Length: 89, dtype: float64

In [117]:
items_bought_by_A = set(customer_matrix.loc[12.0].iloc[customer_matrix.loc[12.0].to_numpy().nonzero()].index)
items_bought_by_A

{28, 31, 33, 34, 35, 41, 43, 67, 68, 73, 75}

In [103]:
items_bought_by_B = set(customer_matrix.loc[17.0].iloc[customer_matrix.loc[17.0].to_numpy().nonzero()].index)
items_bought_by_B

{11, 13, 26, 31, 41, 53, 59, 75, 76}

In [104]:
items_to_recommend_User_B = items_bought_by_A - items_bought_by_B
items_to_recommend_User_B

{28, 33, 34, 35, 43, 67, 68, 73}

In [118]:
df1.loc[
    df['productid'].isin(items_to_recommend_User_B),
    ['productid','productname']
].drop_duplicates().set_index('productid')

,productname
productid,
33,Product ASTMN
35,Product NEVTJ
43,Product ZZZHR
28,Product OFBNT
73,Product WEUJZ
67,Product XLXQF
34,Product SWNJY
68,Product TBTBL


In [120]:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_matrix.T))

In [121]:
item_item_sim_matrix.columns = customer_matrix.T.index
item_item_sim_matrix['productid'] = customer_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('productid')
item_item_sim_matrix.head()

productid,1,2,3,4,5,6,7,8,9,10,...,68,69,70,71,72,73,74,75,76,77
productid,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.475191,0.283981,0.435607,0.179605,0.270765,0.200805,0.348695,0.240966,0.528352,...,0.380216,0.403280,0.459078,0.449346,0.312653,0.216612,0.379071,0.468979,0.406449,0.305480
2,0.475191,1.000000,0.298807,0.366679,0.314970,0.227921,0.338062,0.314485,0.338062,0.444750,...,0.400066,0.462910,0.517549,0.509175,0.460566,0.455842,0.398862,0.559259,0.427669,0.321429
3,0.283981,0.298807,1.000000,0.230089,0.210819,0.381385,0.424264,0.350823,0.141421,0.372104,...,0.182574,0.129099,0.346410,0.365148,0.385337,0.190693,0.190693,0.275241,0.330289,0.239046
4,0.435607,0.366679,0.230089,1.000000,0.242536,0.365636,0.271163,0.403604,0.325396,0.380521,...,0.280056,0.247537,0.221404,0.513436,0.379980,0.292509,0.292509,0.379980,0.379980,0.366679
5,0.179605,0.314970,0.210819,0.242536,1.000000,0.100504,0.372678,0.092450,0.149071,0.196116,...,0.128300,0.204124,0.060858,0.256600,0.464207,0.201008,0.100504,0.174078,0.174078,0.188982


In [108]:
top_10_similar_items = list(
    item_item_sim_matrix\
        .loc[33]\
        .sort_values(ascending=False)\
        .iloc[:10]\
    .index
)
top_10_similar_items

[33, 75, 70, 35, 72, 26, 62, 43, 59, 54]

In [109]:
df.loc[
    df['productid'].isin(top_10_similar_items), 
    ['productid', 'productname']
].drop_duplicates().set_index('productid').loc[top_10_similar_items]

,productname
productid,
33,Product ASTMN
75,Product BWRLG
70,Product TOONT
35,Product NEVTJ
72,Product GEEOO
26,Product HLGZA
62,Product WUXYK
43,Product ZZZHR
59,Product UKXRI
